In [1]:
import requests
from lxml import etree
import csv
import fake_useragent
import threading
 
ua = fake_useragent.UserAgent()  # ua伪装

In [2]:
# 代理池
import requests

PROXY_POOL_URL = 'http://localhost:5555/random'
proxies = ""


def get_proxy():
    try:
        response = requests.get(PROXY_POOL_URL)
        if response.status_code == 200:
            return response.text
    except ConnectionError:
        return None
    
# print(get_proxy())

In [3]:
website = "https://www.shicimingju.com/chaxun/zuozhe/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62'}
i = 1  # 诗人
j = 1  # 诗页
count = 0  # 诗人诗歌数

In [4]:
# 定义需要的列表
poets_list = []
titles_list = []
dynasty_list = []
types_list = []
introductions_list = []
content_list = []
count_list = []

types_set_list = []  # 去重
dynasty_set_list = []  # 去重

In [13]:
# 建立csv节点文件
f_poets = open('./poets.csv', 'w', newline="")
f_dynasty = open('./dynasty.csv', 'w', newline="")
f_poems = open('./poems.csv', 'w', newline="")
f_types = open('./types.csv', 'w', newline="")

writer_poets = csv.writer(f_poets)
writer_poets.writerow(['index:name', 'introduction', ':LABEL'])

writer_dynasty = csv.writer(f_dynasty)
writer_dynasty.writerow(['index:dynasty', ':LABEL'])

writer_poems = csv.writer(f_poems)
writer_poems.writerow(['index:title', 'content', ':LABEL'])

writer_types = csv.writer(f_types)
writer_types.writerow(['index:type', ':LABEL'])



# 建立csv关系文件
f_poets_dynasty = open('./poets_dynasty.csv', 'w', newline="")
f_poems_dynasty = open('./poems_dynasty.csv', 'w', newline="")
f_poets_poems = open('./poets_poems.csv', 'w', newline="")
f_poems_types = open('./poems_types.csv', 'w', newline="")

writer_poets_dynasty = csv.writer(f_poets_dynasty)
writer_poets_dynasty.writerow([':START_name', ':END_dynasty', 'relation', ':TYPE'])

writer_poems_dynasty = csv.writer(f_poems_dynasty)
writer_poems_dynasty.writerow([':START_title', ':END_dynasty', 'relation', ':TYPE'])

writer_poets_poems = csv.writer(f_poets_poems)
writer_poets_poems.writerow([':START_name', ':END_title', 'relation', ':TYPE'])

writer_poems_types = csv.writer(f_poems_types)
writer_poems_types.writerow([':START_title', ':END_type', 'relation', ':TYPE'])

39

In [11]:
# 多线程技术
import threading
import time
 
# 这个函数名可随便定义
def get_ip():
    while True:
        if get_proxy() is not None:
            proxies = get_proxy()
            time.sleep(1)
            print(proxies)
        else:
            pass

t1 = threading.Thread(target=get_ip)
t1.start()

118.31.2.38:8999
157.230.34.219:3128


In [12]:
# 主爬取&写入函数
def main():
    global i
    global j
    global count
    while 1:

        try:  # 诗人已经到达尽头，可以退出爬取了

            # 网页构建
            web = website + str(i) + "_" + str(j) + ".html"

            # 古诗列表模式
            try:
                response = requests.get(url=web, headers={'User-Agent': ua.random}, proxies={"http":proxies})
            except:
                response = requests.get(url=web, headers={'User-Agent': ua.random})
            page_text = response.content  # content防止中文乱码问题
            tree = etree.HTML(page_text)

            poet = tree.xpath('//*[@id="main_right"]/div[1]/div[2]/div[1]/h4/a/text()')[0]  # 诗人
            dynasty = tree.xpath('//*[@id="main_right"]/div[1]/div[3]/div[1]/div[2]/a/text()')[0]  # 朝代

            try:  # 诗人介绍有可能不存在
                # 诗人介绍
                poet_intr = tree.xpath('//*[@id="main_right"]/div[1]/div[2]/div[1]/div/text()')
                intr_max = ""
                for intr in poet_intr:
                    intr_max += intr
                intr_max = intr_max.strip()
            except:
                intr_max = ""

            try:
                # 诗歌总页数的列表
                poet_all = tree.xpath('//*[@id="list_nav_all"]/a')
                poet_num = len(poet_all)
            except:
                poet_num = 1

            # 诗文列表
            poet_list = tree.xpath('//*[@id="main_left"]/div[1]/div')

            for poet_pre in poet_list[1::2]:  # 跳一个取一个
                title = poet_pre.xpath('./div[2]/h3/a/text()')[0]  # 标题

                # 古诗详情页
                poet_url = "https://www.shicimingju.com" + poet_pre.xpath('./div[2]/h3/a/@href')[0]
                # 进入具体古诗页面
                try:
                    response = requests.get(url=poet_url, headers={'User-Agent': ua.random}, proxies={"http":proxies})
                except:
                    response = requests.get(url=poet_url, headers={'User-Agent': ua.random})
                    
                page_text = response.content  # content防止中文乱码问题
                tree = etree.HTML(page_text)

                # 诗歌内容
                content = tree.xpath('//*[@id="zs_content"]/text()')
                content_max = ""
                for con in content:
                    content_max += con
                content_max = content_max.strip()

                # 诗歌标签
                try:
                    labels = tree.xpath('//*[@id="item_div"]/div[5]/a/text()')
                except:
                    labels = ["暂无"]

                # 循环赋值
                poets_list.append(poet)
                titles_list.append(title)
                dynasty_list.append(dynasty)
                types_list.append(labels)
                introductions_list.append(intr_max)
                content_list.append(content_max)
                count += 1  # 计数


                # 写入文件
                for label in labels:  # 去重类型
                    if label not in types_set_list:
                        types_set_list.append(label)
                        writer_types.writerow([label, '类型'])
                    writer_poems_types.writerow([title, label, 'belongto', '属于'])

                # 去重朝代
                if dynasty not in dynasty_set_list:
                    dynasty_set_list.append(dynasty)
                    writer_dynasty.writerow([dynasty, '朝代'])

                if j == 1:  # 诗人变更
                    writer_poets.writerow([poet, intr_max, "诗人"])
                    writer_poets_dynasty.writerow([poet, dynasty, 'livein', '生活于'])

                writer_poems.writerow([title, content_max, '诗词'])
                writer_poems_dynasty.writerow([title, dynasty, 'writenin', '创作于'])
                writer_poets_poems.writerow([poet, title, 'write', '写作'])

                print("{0}的{1}爬取成功！".format(poet, title))

            # 判断条件
            if j < poet_num:
                j += 1  # 循环完成一页
                print("-----------------------------第{0}页爬取完毕--------------------------------".format(j - 1))
            else:  # 此诗人循环完毕，进行下一个诗人爬取
                i += 1  # 诗人切换
                j = 1  # 页数归零
                print("============================================================================")
                print("{0}爬取完毕，共获得{1}首诗歌！！".format(poet, count))
                print("============================================================================")
                count_list.append("count")
                count = 0  # 计数清零

        except Exception as e:
            print()
            print(e)
            break

            
t2 = threading.Thread(target=main)  # , args=("thread 2",))
t2.start()

182.241.132.30:80
苏轼的《西江月　送别》爬取成功！
183.221.242.107:8443
苏轼的《江神子/江城子》爬取成功！
110.34.3.229:3128
苏轼的《江神子/江城子》爬取成功！
173.212.200.30:3128
苏轼的《江神子/江城子》爬取成功！
110.34.3.229:3128
苏轼的《江神子/江城子》爬取成功！
122.155.165.191:3128
苏轼的《江神子/江城子》爬取成功！
110.34.3.229:3128
苏轼的《古意》爬取成功！
苏轼的《浣溪沙 咏橘》爬取成功！
105.112.191.250:3128
苏轼的《桃源忆故人·华胥梦断人何处》爬取成功！
122.9.21.228:8000
苏轼的《天仙子·走马探花花发未》爬取成功！
182.241.132.30:80
苏轼的《西江月　平山堂》爬取成功！
苏轼的《竹》爬取成功！
66.94.108.138:3128
苏轼的《阮郎归 初夏》爬取成功！
110.34.3.229:3128

'gbk' codec can't encode character '\u2022' in position 4: illegal multibyte sequence
66.94.108.138:3128
118.31.2.38:8999
122.9.21.228:8000
105.112.191.250:3128
182.241.132.30:80
182.241.132.30:80
183.221.242.107:8443


In [ ]:
# 古诗详情页
poet_url = "https://www.shicimingju.com/chaxun/list/109357.html"
# 进入具体古诗页面
response = requests.get(url=poet_url, headers={'User-Agent': ua.random}, proxies={"http":get_proxy()})
page_text = response.content  # content防止中文乱码问题
tree = etree.HTML(page_text)

# 诗歌内容
content = tree.xpath('//*[@id="zs_content"]/text()')
content_max = ""
for con in content:
    content_max += con
content_max = content_max.replace("\r", "").replace("\n", "").replace(" ", "")
# content_max = content_max.strip()
print(content_max)

In [ ]:
while 1:

    try:  # 诗人已经到达尽头，可以退出爬取了

        # 网页构建
        web = website + str(i) + "_" + str(j) + ".html"

        # 古诗列表模式
        if get_proxy() is not None:
            response = requests.get(url=web, headers={'User-Agent': ua.random}, proxies={"http":get_proxy()})
        else:
            response = requests.get(url=web, headers={'User-Agent': ua.random})
        page_text = response.content  # content防止中文乱码问题
        tree = etree.HTML(page_text)

        poet = tree.xpath('//*[@id="main_right"]/div[1]/div[2]/div[1]/h4/a/text()')[0]  # 诗人
        dynasty = tree.xpath('//*[@id="main_right"]/div[1]/div[3]/div[1]/div[2]/a/text()')[0]  # 朝代

        try:  # 诗人介绍有可能不存在
            # 诗人介绍
            poet_intr = tree.xpath('//*[@id="main_right"]/div[1]/div[2]/div[1]/div/text()')
            intr_max = ""
            for intr in poet_intr:
                intr_max += intr
            intr_max = intr_max.strip()
        except:
            intr_max = ""

        try:
            # 诗歌总页数的列表
            poet_all = tree.xpath('//*[@id="list_nav_all"]/a')
            poet_num = len(poet_all)
        except:
            poet_num = 1

        # 诗文列表
        poet_list = tree.xpath('//*[@id="main_left"]/div[1]/div')

        for poet_pre in poet_list[1::2]:  # 跳一个取一个
            title = poet_pre.xpath('./div[2]/h3/a/text()')[0]  # 标题

            # 古诗详情页
            poet_url = "https://www.shicimingju.com" + poet_pre.xpath('./div[2]/h3/a/@href')[0]
            # 进入具体古诗页面
            if get_proxy() is not None:
                response = requests.get(url=poet_url, headers={'User-Agent': ua.random}, proxies={"http":get_proxy()})
            else:
                response = requests.get(url=poet_url, headers={'User-Agent': ua.random})
            page_text = response.content  # content防止中文乱码问题
            tree = etree.HTML(page_text)

            # 诗歌内容
            content = tree.xpath('//*[@id="zs_content"]/text()')
            content_max = ""
            for con in content:
                content_max += con
            content_max = content_max.strip()

            # 诗歌标签
            try:
                label = tree.xpath('//*[@id="item_div"]/div[5]/a/text()')
            except:
                label = ["暂无"]

            # 循环赋值
            poets_list.append(poet)
            titles_list.append(title)
            dynasty_list.append(dynasty)
            types_list.append(label)
            introductions_list.append(intr_max)
            content_list.append(content_max)
            count += 1  # 计数
            print("{0}的{1}爬取成功！".format(poet, title))

        # 判断条件
        if j < poet_num:
            j += 1  # 循环完成一页
            print("-----------------------------第{0}页爬取完毕--------------------------------".format(j - 1))
        else:  # 此诗人循环完毕，进行下一个诗人爬取
            i += 1  # 诗人切换
            j = 1  # 页数归零
            print("============================================================================")
            print("{0}爬取完毕，共获得{1}首诗歌！！".format(poet, count))
            print("============================================================================")
            count_list.append("count")
            count = 0  # 计数清零

    except Exception as e:
        print(e)
        break
